# Example 2: Computation of Gradients and Non-adiabatic Couplings

In this tutorial it is shown, how to use the SA-OO-VQE solver in a simple step-by-step manner on a molecule of
formaldimine (methylene imine). It is shown, how to compute energies, gradients and non-adiabatic couplings
in several different settings w.r.t. a different number of optimized orbitals.

First of all, we'll specify geometry of the system.

In [2]:
symbols = ['N', 'C', 'H', 'H', 'H']
coords = [
    [0.000000000000, 0.000000000000, 0.000000000000],
    [0.000000000000, 0.000000000000, 1.498047000000],
    [0.000000000000, -0.938765985000, 2.004775984000],
    [0.000000000000, 0.938765985000, 2.004775984000],
    [-0.744681452, -0.131307432, -0.634501434]
]

Now we'll specify its properties, active space and the basis for Psi4 chemistry backend.

In [3]:
n_orbs_active = 2
n_elec_active = 2
charge = 0
multiplicity = 1
basis = 'sto-3g'

The next step is construction of `ProblemSet` instance - object containing all the information and necessary method for
our electronic structure problem.

In [4]:
import saoovqe

problem = saoovqe.problem.ProblemSet(
    symbols=symbols,
    coords=coords,
    charge=charge,
    multiplicity=multiplicity,
    n_electrons_active=n_elec_active,
    n_orbitals_active=n_orbs_active,
    basis_name=basis
)

Now we need to create a set of circuits representing orthogonal states to construct the whole circuits representing
state vectors later.

In [5]:
initial_circuits = saoovqe.OrthogonalCircuitSet.from_problem_set(n_states=2, problem=problem)

The next necessary part is to define an ansatz - it'll be also used to construct the state vector circuits later.

In [6]:
ansatz = saoovqe.Ansatz.from_problem_set(ansatz=saoovqe.AnsatzType.GUCCSD,
                                         problem=problem,
                                         repetitions=1,
                                         qubit_mapper=problem.fermionic_mapper)

And finally, now we can create an instance of our SA-OO-VQE solver. One of the main points is, orbital-optimization
can, but doesn't have to be used, or it can be used only on some molecular orbitals. We'll show all three cases here.
For no orbital-optimization it's enough to pass `None` to `orbital_optimization_settings` (it's also a default value).

In [7]:
from qiskit.primitives import Estimator, Sampler

estimator = Estimator()
sampler = Sampler()

solver_no_oo = saoovqe.SAOOVQE(estimator=estimator,
                               initial_circuits=initial_circuits,
                               ansatz=ansatz,
                               problem=problem,
                               orbital_optimization_settings=None)

To specify number of optimized orbitals we can pass a dictionary to the parameter.

In [8]:
solver_oo_8 = saoovqe.SAOOVQE(estimator=estimator,
                              initial_circuits=initial_circuits,
                              ansatz=ansatz,
                              problem=problem,
                              orbital_optimization_settings={'n_mo_optim': 8})

And for all the orbitals to be optimized we can simply use its default behavior by passing an empty dictionary.

In [9]:
solver_oo_full = saoovqe.SAOOVQE(estimator=estimator,
                                 initial_circuits=initial_circuits,
                                 ansatz=ansatz,
                                 problem=problem,
                                 orbital_optimization_settings={})

Let's compare the energies now! To make the numerical optimizations, we'll use `SLSQP` optimizer provided by `SciPy`.
It may take a few minutes time now...

In [10]:
from qiskit_algorithms.optimizers import SciPyOptimizer
import numpy as np

optimizer = SciPyOptimizer('SLSQP', options={'maxiter': 500, 'ftol': 1e-8})
energies_no_oo = solver_no_oo.get_energy(optimizer)
energies_oo_8 = solver_oo_8.get_energy(optimizer)
energies_oo_full = solver_oo_full.get_energy(optimizer)

print('\n============== State-Averaged Energies ==============')
print(np.mean(energies_no_oo))
print(np.mean(energies_oo_8))
print(np.mean(energies_oo_full))

And now, let's have a look at the gradients of the potential energy surface for all the particles at both relevant
states.

In [11]:
print('\n============== Gradients ==============')
for state_idx in range(2):
    for atom_idx in range(len(symbols)):
        print(state_idx, atom_idx, solver_oo_full.eval_eng_gradient(state_idx, atom_idx))

And the non-adiabatic couplings.

In [12]:
print('\n============== Total non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.eval_nac(atom_idx))

And finally, considering NACs, we can also have a look at CI and CSF NACs separately.

In [13]:
print('\n============== CI non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.ci_nacs[atom_idx])

print('\n============== CSF non-adiabatic couplings ==============')
for atom_idx in range(len(symbols)):
    print(atom_idx, solver_oo_full.csf_nacs[atom_idx])